<a href="https://colab.research.google.com/github/sunhuanhuan920/TimeSeries/blob/main/TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Time Warping (DTW)

In [1]:
import numpy as np

In [4]:
def dtw(x, x_prime):

    R = np.zeros((len(x), len(x_prime)))

    for i in range(len(x)):
        for j in range(len(x_prime)):
            R[i, j] = np.sqrt((x[i] - x_prime[j])**2)
            if i > 0 or j > 0:
                R[i, j] += min(R[i-1, j] if i > 0 else np.inf,
                               R[i, j-1] if j > 0 else np.inf,
                               R[i-1, j-1] if i > 0 and j > 0 else np.inf)

    return R

In [7]:
# Time series x and x'
x = [7, 1, 2, 5, 9]
x_prime = [1, 8, 0, 4, 4, 2, 0]

print(np.flipud(dtw(x, x_prime)))

[[19. 11. 19. 16. 16. 18. 23.]
 [11. 10. 15. 11. 11. 14. 17.]
 [ 7. 12. 10. 10. 12. 12. 14.]
 [ 6. 13.  8. 11. 14. 15. 16.]
 [ 6.  7. 14. 17. 20. 25. 32.]]
